In [ ]:
%run setup_Sv_compute.ipynb

# Compute Sv, for OS samples at first 

In [89]:
pd.options.display.max_rows = 999
dfnow = df[df['geo_region'].notnull()]
dfnow = dfnow[dfnow['geo_region'].str.contains('acific')]
#dfnow[dfnow['geo_region'].str.contains('acific')][['fname','geo_region']]
# --> I like "central equatorial pacific" --> choose 00890_short.nc
#fnamenow = '00890_short.nc'
#fnamenow = '00249_short.nc'
fnamenow = '01006_short.nc'
attrsnow = df[df['fname']==fnamenow]

In [90]:
ncfile = dpath + 'JASADCP/ncfiles/' + fnamenow
nc = xr.open_dataset(ncfile)
#strnow = nc.attrs['cruise_sonar_summary']
#re.findall('arro', strnow, re.IGNORECASE)
nc.attrs['cruise_sonar_summary'].split('\n')
#nc
#print(nc['depth'])
#nc['amp']
#nc['amp'].plot()
#nc['depth'].values
#nc['ADCP_CONFIG'].attrs['model_name']
#nc['ADCP_CONFIG'].attrs

[' #DATA_DATES: 2001/09/10 15:26:00 --- to --- 2001/10/25 10:47:00',
 ' #LON_RANGE: 101.50 W --- to ---  70.35 W',
 ' #LAT_RANGE:  20.31 S --- to ---  13.62 N',
 ' #DEPTH_RANGE:     13 --- to --- 489 m',
 ' #SAC_CRUISE_ID: 01006 ',
 ' #PLATFORM_NAME: R/V Ron Brown',
 ' #PRINCIPAL_INVESTIGATOR_NAME: E.Firing, J.Hummon',
 ' #PI_INSTITUTION:  University of Hawaii',
 ' #PI_COUNTRY:      USA',
 ' #PROJECT: STRATUS, stratus clouds of the eastern tropical Pacific',
 ' #CRUISE_NAME: RB0108',
 ' #PORTS: San Diego to Galapagos, Galapagos to Arica, Chile',
 ' #GEOGRAPHIC_REGION: eastern tropical and southeast Pacific',
 ' #PROCESSED_BY: University of Hawaii',
 ' #NAVIGATION: GPS ',
 ' #QUALITY_NAV: good (no Ashtech)',
 ' #GENERAL_INFORMATION: ',
 'CRUISE NOTES',
 '  CHIEF SCIENTIST ON SHIP     : unconfirmed',
 '    INSTITUTE                 : unconfirmed',
 '  SIGNIFICANT DATA GAPS       : many gaps during first leg of cruise',
 '  SPECIAL SHIP TRACK PATTERNS : ',
 '  COMMENTS                    

In [ ]:
def calc_c_tdresolved(T=25,S=35,z):
# returns time-depth resolved c, sound speed in m/s
    return c
def calc_alpha_tdresolved(T,S,z):
# returns time-depth alpha, absorption coefficient in dB/m
    return alpha

In [ ]:
def get_C_constant(row):
# returns constant C in dB
    if row['bandwidth']=='narrowband':
        Cnow = 'C_6%'
    elif row['bandwidth']=='broadband':
        Cnow = 'C_25%'
        
    if (re.search('OS', row['instrument_name'])):
        if (re.search('OS-II'), row['instrument_name']):
            instrument_prefixnow = 'OS-II'
        else:
            instrument_prefixnow = 'OS'
        return OS_C_PDBW_Kc_Er_values[
            (OS_C_PDBW_Kc_Er_values['instrument_prefix']==instrument_prefixnow)
            & (OS_C_PDBW_Kc_Er_values['freq_round_kHz']==row['freq_round_kHz'])
            ][Cnow]
    elif (re.search('WH', row['instrument_name'])):
        if (re.search('WH-Mariner'), row['instrument_name']):
            instrument_prefixnow = 'WH-Mariner'
        else:
            instrument_prefixnow = 'WH'
        return WH_C_PDBW_Kc_Er_values[
            (WH_C_PDBW_Kc_Er_values['instrument_prefix']==instrument_prefixnow)
            & (WH_C_PDBW_Kc_Er_values['freq_round_kHz']==row['freq_round_kHz'])
            ][Cnow]
    elif (re.search('BB', row['instrument_name'])):
        instrument_prefixnow = 'BB-VM'
        return BB_C_PDBW_Kc_Er_values[
            (BB_C_PDBW_Kc_Er_values['instrument_prefix']==instrument_prefixnow)
            & (BB_C_PDBW_Kc_Er_values['freq_round_kHz']==row['freq_round_kHz'])
            ][Cnow]
    
def get_C_tdresolved(row,c):
# returns time-depth resolved C in dB
    if (re.search('NB'), row['instrument_name']):
        if (re.search('NB-VM', row['instrument_name'])):
            instrument_prefixnow = 'NB-VM'
        elif (re.search('NB-DR', row['instrument_name'])):
            instrument_prefixnow = 'NB-DR'
        K2 = NB_K2_values[(NB_K2_values['instrument_prefix']==instrument_prefixnow)
                          & (NB_K2_values['freq_round_kHz']==row['freq_round_kHz'])
                         ]['K2']
        Ks = NB_Ks_values[(NB_Ks_values['freq_round_kHz']==row['freq_round_kHz'])
                         ]['Ks']
        C = 10*np.log10(4.47*10^(-20)*K2*Ks/c)
        return C

In [ ]:
def get_R_tdresolved(row, depth, method_num=5, c_wtavg=1475.1): # m
# return time-depth resolved R in m
    if method_num==4:
        return depth
    if method_num==5:
        return depth/np.cos(row['theta']*np.pi/180)
    if method_num==6:
        # c_wtavg can be time-depth resolved, just like depth
        return depth/np.cos(row['theta']*np.pi/180)*c_wtavg/1475.1

$S_v = \color{red}{C} + \color{blue}{10log((T_x + 273.16)*R^2)} - \color{orange}{L_{DBM}} - \color{green}{P_{DBW}} + \color{purple}{2 \alpha R} + 10log(10^{k_c(E-E_r)/10} - 1)$  
$S_v = \color{red}{10log(\frac{4.47 \times 10^{-20} K_2 K_S}{c})} + \color{blue}{10log((T_x + 273.16)*R^2)} - \color{orange}{10log(P)} - \color{green}{10log(K_1)} + \color{purple}{2 \alpha R} + 10log(10^{k_c(E-E_r)/10} - 1)$

In [ ]:
def get_LDBM_tdresolved(row, 
    return np.log10(row['transmit_pulse_length'])

In [82]:
np.log10(24)-np.log10(8)

0.4771212547196624

In [96]:
dfnow = df[df['transmit_pulse_length'].notnull()]
#dfnow[dfnow['transmit_pulse_length'].str.contains('varies')]['fname']
dfnow[dfnow['transmit_pulse_length']=='??']['fname']

306    00342_short.nc
307    00343_short.nc
308    00344_short.nc
309    00345_short.nc
Name: fname, dtype: object

In [95]:
# - Standardize transmit_pulse_length to a number
pd.options.display.max_rows = 999
#print(df['transmit_pulse_length'].value_counts())
dfgroupnow = df.groupby(['instrument_name','transmit_pulse_length'])
#dfgroupnow = df.groupby(['instrument_name','bandwidth','transmit_pulse_length'])
print(dfgroupnow['fname'].nunique())

instrument_name  transmit_pulse_length                
BB-VM-150        2 m                                        1
                 8 m                                        1
                 8.08 m                                     6
                 9.32 m                                     5
NB-DR-150        4 m                                        3
NB-VM-150        12 m                                       4
                 16 m                                     185
                 16 m (4 briefly at beginning and end)      1
                 16 m (4 m at times on shelf)              10
                 16 m (at times 24 or 32)                   1
                 16m                                        5
                 2, 4, or 8m                                1
                 4 - 12 m                                   1
                 4 - 16 m                                  26
                 4 - 6 m                                   12
               

In [ ]:
def get_Kc_constant(row):
# returns constant Kc in dB/count
    if (re.search('OS', row['instrument_name'])):
        if (re.search('OS-II'), row['instrument_name']):
            return OS_C_PDBW_Kc_Er_values[
                (OS_C_PDBW_Kc_Er_values['instrument_prefix']=='OS-II')
                & (OS_C_PDBW_Kc_Er_values['freq_round_kHz']==row['freq_round_kHz'])
                ]['Kc_mean']
        else:
            return OS_C_PDBW_Kc_Er_values[
                (OS_C_PDBW_Kc_Er_values['instrument_prefix']=='OS')
                & (OS_C_PDBW_Kc_Er_values['freq_round_kHz']==row['freq_round_kHz'])
                ]['Kc_mean']
    elif (re.search('WH', row['instrument_name'], re.IGNORECASE)):
        if (re.search('WH-Mariner'), row['instrument_name']):
            return WH_C_PDBW_Kc_Er_values[
                (WH_C_PDBW_Kc_Er_values['instrument_prefix']=='WH-Mariner')
                & (WH_C_PDBW_Kc_Er_values['freq_round_kHz']==row['freq_round_kHz'])
                ]['Kc_mean']
        else:
            return WH_C_PDBW_Kc_Er_values[
                (WH_C_PDBW_Kc_Er_values['instrument_prefix']=='WH')
                & (WH_C_PDBW_Kc_Er_values['freq_round_kHz']==row['freq_round_kHz'])
                ]['Kc_mean']
    elif (re.search('BB', row['instrument_name'], re.IGNORECASE)):
        return BB_C_PDBW_Kc_Er_values[
            (BB_C_PDBW_Kc_Er_values['instrument_prefix']=='BB-VM')
            & (BB_C_PDBW_Kc_Er_values['freq_round_kHz']==row['freq_round_kHz'])
            ]['Kc_mean']

def get_Kc_tdresolved(row,amp,Tx):
# returns time-depth resolved Kc in dB/count
# this assumes that Tx = temp of transducer = Te = temp of system electronics
    if (re.search('NB', row['instrument_name'])):
        if amp < 200:
            Kcnow = 127.3/(Tx+271)
        else:
            Kcnow = None
        return Kcnow

# Load WOD data (T,S)

# TESTING/OLD

In [ ]:
fnames = sorted(os.listdir(dpath + 'JASADCP/ncfiles'))

In [ ]:
nc_counter = len(fnames)
geo_region = [None]*nc_counter

In [ ]:
ifile = 0
for fname in fnames:
    ncfile = dpath + 'JASADCP/ncfiles/' + fname
    ncnow = xr.open_dataset(ncfile)
    strnow = ncnow.attrs['cruise_sonar_summary']
    geo_regionnow = re.findall("GEOGRAPHIC_REGION *: *((?:\S+ )*\S+)", strnow)
    if geo_regionnow:
        geo_region[ifile] = geo_regionnow[0]
    ifile = ifile+1

In [ ]:
df = pd.concat(
    [pd.Series(fnames,name='fname'), pd.Series(geo_region,name='geo_region')],
    axis=1)

In [ ]:
print(df['geo_region'].isnull().sum())
print(len(df))
dfnow = df.dropna(subset=['geo_region'])
print(len(dfnow))
dfnow = dfnow[dfnow['geo_region'].str.contains('acific')]
print(len(dfnow))
dfnow = dfnow[dfnow['instrument_name'].str.contains('OS')]
print(len(dfnow))
dfnow = dfnow[(dfnow['instrument_name']!='OS-II-38')&(dfnow['instrument_name']!='OS-II-75')] 

## Load table of C, PDBW values

In [ ]:
cpdbw = pd.read_csv(dpath + 'typical_system_characteristics_table2_mullison2017.csv')

In [ ]:
cpdbw.head()

## Load table of kc, Er values

#### Nominal Kc values from TRDI field service email:
BBADCP: 0.45 dB/count  
WHADCP: 0.42 dB/count  
OS-II-75, SN1508: 0.373 (beam 1), 0.386 (beam 2), 0.388 (beam 3), 0.384 (beam 4) = 0.38275 (avg)  
OS-II-75, SN10656: 0.398 (beam 1), 0.389 (beam 2), 0.3988 (beam 3), 0.395 (beam 4) = 0.395 (avg)  
#### Nominal Kc values from Jerry Mullison email:
Format: min-max, avg, stdev  
OS150: 0.41-0.44, 0.42, 0.012   
OS75: 0.36-0.42, 0.39, 0.019  
OS38: 0.36-0.37, 0.37, 0.004  

#### Nominal Er values from Jerry Mullison email:
Format: min-max, avg, stdev  
OS150: 18-28, 22, 2.08  
OS75: 11-25, 19, 2.93  
OS38: 5-33, 14, 4.83  

In [ ]:
# - Calc avg Kc for OS-II w/ SNs
print(np.mean([0.373, 0.386, 0.388, 0.384]))
print(np.mean([0.398, 0.389, 0.398, 0.395]))